# Importing necessary libraries

In [2]:
import sys
import numpy as np
import pandas
from astropy import constants as const
from astropy import units as u
from astropy.io import fits
from astropy.io import ascii
import random
import matplotlib.pyplot as plt
import wget
import seaborn as sns
#import healpy

## Import and Process Bricks Summary

In [9]:
#Open fits images
hdulistBricksSummary = fits.open('../bricks_data/survey-bricks-dr9-south.fits')
dataSummary = hdulistBricksSummary[1].data
bricknameSummary = dataSummary.field('brickname')
raSummary = dataSummary.field('ra')
decSummary = dataSummary.field('dec')
brickIDSummary = dataSummary.field('brickid')
nobjsSummary = dataSummary.field('nobjs')
nexp_gSummary = dataSummary.field('nexp_g')
nexp_rSummary = dataSummary.field('nexp_r')
nexp_zSummary = dataSummary.field('nexp_z')

raDec = np.stack((raSummary, decSummary,nexp_gSummary,nexp_rSummary,nexp_zSummary), axis=-1)
print(raSummary)
print(decSummary)
print(raDec.shape)

print(nexp_gSummary)
print(raDec[:,2])


[1.25000000e-01 1.25000000e-01 1.25000000e-01 ... 3.59850000e+02
 3.59849498e+02 3.59849246e+02]
[-0.25 -0.5  -0.75 ... 33.75 34.   34.25]
(253658, 5)
[8 8 9 ... 2 1 0]
[8. 8. 9. ... 2. 1. 0.]


In [10]:
print(bricknameSummary[49956])
print(raSummary)
print(decSummary)
print(brickIDSummary)
print(nobjsSummary)
print(nexp_gSummary)

0374m195
[1.25000000e-01 1.25000000e-01 1.25000000e-01 ... 3.59850000e+02
 3.59849498e+02 3.59849246e+02]
[-0.25 -0.5  -0.75 ... 33.75 34.   34.25]
[328928 327488 326048 ... 515285 516481 517675]
[7986 8746 8422 ... 3867 3505  625]
[8 8 9 ... 2 1 0]


In [8]:
brickID = '0374m195'
folder = brickID[:3]
url = f'https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr9/south/tractor/{folder}/tractor-{brickID}.fits'
wget.download(url, '../bricks_data/tractor/')

'../bricks_data/tractor//tractor-0374m195.fits'

In [14]:
url = 'https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr9/south/tractor/000/tractor-0001m002.fits'
wget.download(url, '../bricks_data/tractor/')


## Importing and Processing a single brick tractor file

In [12]:
hdulistSingleBrick = fits.open(f'../bricks_data/tractor/tractor-{brickID}.fits')
hdulistSingleBrick.info()

Filename: ../bricks_data/tractor/tractor-0374m195.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     225   ()      
  1                1 BinTableHDU    447   8923R x 181C   [I, J, 8A, J, L, I, I, 3A, D, D, E, E, E, E, 5E, E, D, D, 2A, K, E, E, E, E, E, E, E, E, E, I, E, E, I, E, E, I, L, E, E, I, I, E, L, E, E, E, E, B, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, 8E, 8E, 8E, 8E, 8E, 8E, 8E, 8E, 8E, 8E, 8E, 8E, 8E, 8E, 8E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, 5E, E, E, E, E, E, E, E, I, I, I, I, I, I, I, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, I, I, I, I, I, I, B, B, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, 8A, E, E, 15E, 15E, 15E, 15E, 15I, 15I, 15E, 15E, 15E, 15E, 15D, 15D, 15I, 15I, E, E, E, E, E, E, E, E]   


In [13]:
data = hdulistSingleBrick[1].data

In [14]:
hdulistSingleBrick[1].columns

ColDefs(
    name = 'release'; format = 'I'
    name = 'brickid'; format = 'J'
    name = 'brickname'; format = '8A'
    name = 'objid'; format = 'J'
    name = 'brick_primary'; format = 'L'
    name = 'maskbits'; format = 'I'
    name = 'fitbits'; format = 'I'
    name = 'type'; format = '3A'
    name = 'ra'; format = 'D'; unit = 'deg'
    name = 'dec'; format = 'D'; unit = 'deg'
    name = 'ra_ivar'; format = 'E'; unit = '1/deg^2'
    name = 'dec_ivar'; format = 'E'; unit = '1/deg^2'
    name = 'bx'; format = 'E'
    name = 'by'; format = 'E'
    name = 'dchisq'; format = '5E'
    name = 'ebv'; format = 'E'; unit = 'mag'
    name = 'mjd_min'; format = 'D'
    name = 'mjd_max'; format = 'D'
    name = 'ref_cat'; format = '2A'
    name = 'ref_id'; format = 'K'
    name = 'pmra'; format = 'E'; unit = 'mas/yr'
    name = 'pmdec'; format = 'E'; unit = 'mas/yr'
    name = 'parallax'; format = 'E'; unit = 'mas'
    name = 'pmra_ivar'; format = 'E'; unit = '1/(mas/yr)^2'
    name = 'pmdec_iv

In [15]:
ID = data.field('brickid')
name = data.field('brickname')
objid = data.field('objid')

print(ID)
print(name)
print(objid)

[220271 220271 220271 ... 220271 220271 220271]
['0374m195' '0374m195' '0374m195' ... '0374m195' '0374m195' '0374m195']
[   0    1    2 ... 8920 8921 8922]


## Getting the Fluxes per object in this brick

## Converting Fluxes to extiction corrected magnitudes
conversion from linear fluxes to magnitudes is m=22.5−2.5log10(flux)

In [16]:
#fluxes and magnitudes
flux_g = data.field('flux_g')
flux_r = data.field('flux_r')
flux_z = data.field('flux_z')
flux_w1 = data.field('flux_w1')
flux_w2 = data.field('flux_w2')
flux_w3 = data.field('flux_w3')
flux_w4 = data.field('flux_w4')

mw_transmission_g = data.field('mw_transmission_g')
mw_transmission_r = data.field('mw_transmission_r')
mw_transmission_z = data.field('mw_transmission_z')
mw_transmission_w1 = data.field('mw_transmission_w1')
mw_transmission_w2 = data.field('mw_transmission_w2')
mw_transmission_w3 = data.field('mw_transmission_w3')
mw_transmission_w4  = data.field('mw_transmission_w4')

#correcting for extinction
flux_g_corrected = flux_g - mw_transmission_g
flux_r_corrected = flux_r - mw_transmission_r
flux_z_corrected = flux_z - mw_transmission_z
flux_w1_corrected = flux_w1 - mw_transmission_w1
flux_w2_corrected = flux_w2 - mw_transmission_w2
flux_w3_corrected = flux_w3 - mw_transmission_w3
flux_w4_corrected = flux_w4 - mw_transmission_w4

mag_g = 22.5-2.5*np.log10(flux_g_corrected)
mag_r = 22.5-2.5*np.log10(flux_r_corrected)
mag_z = 22.5-2.5*np.log10(flux_z_corrected)
mag_w1 = 22.5-2.5*np.log10(flux_w1_corrected)
mag_w2 = 22.5-2.5*np.log10(flux_w2_corrected)
mag_w3 = 22.5-2.5*np.log10(flux_w3_corrected)
mag_w4 = 22.5-2.5*np.log10(flux_w4_corrected)

#Retrieving the maskbits for quasar detection

maskbits = data.field('maskbits')
print(maskbits)

[   1    1 2049 ...    1    1    1]


<ipython-input-16-c643764dfa60>:27: RuntimeWarning: invalid value encountered in log10
  mag_g = 22.5-2.5*np.log10(flux_g_corrected)
<ipython-input-16-c643764dfa60>:28: RuntimeWarning: invalid value encountered in log10
  mag_r = 22.5-2.5*np.log10(flux_r_corrected)
<ipython-input-16-c643764dfa60>:29: RuntimeWarning: invalid value encountered in log10
  mag_z = 22.5-2.5*np.log10(flux_z_corrected)
<ipython-input-16-c643764dfa60>:30: RuntimeWarning: invalid value encountered in log10
  mag_w1 = 22.5-2.5*np.log10(flux_w1_corrected)
<ipython-input-16-c643764dfa60>:31: RuntimeWarning: invalid value encountered in log10
  mag_w2 = 22.5-2.5*np.log10(flux_w2_corrected)
<ipython-input-16-c643764dfa60>:32: RuntimeWarning: invalid value encountered in log10
  mag_w3 = 22.5-2.5*np.log10(flux_w3_corrected)
<ipython-input-16-c643764dfa60>:33: RuntimeWarning: invalid value encountered in log10
  mag_w4 = 22.5-2.5*np.log10(flux_w4_corrected)


Preliminary Target Selection for the DESI Quasar (QSO) Sample

We restrict both selections to objects with stellar morphology, to avoid an almost 10-fold
contamination by galaxies, and we impose a depth limit of r = 22.7 (AB). We also require that the targets are not in
corrupted imaging pixels, nor pixels that are in the vicinity of bright stars, globular clusters, or nearby galaxies. Such
“masked” sources have MASKBITS of 1, 5, 6, 7, 10, 12 or 13 set in Legacy Surveys catalogs.
Our color-cut selection uses W1 − W2 to select redder sources, and r−W vs. g − z (where W is a weighted average
of W1 and W2 fluxes with flux(W)=0.75×flux(W1)+0.25×flux(W2)) to reject stars based on the “infrared excess”
of QSOs (see Figure 1). We impose g − r < 1.3 and −0.4 < r − z < 1.1 to avoid regions in color space almost solely
populated by stars. We also restrict the magnitude range to r > 17.5 and grz > 17.0, where grz is a weighted average
of the three band fluxes (flux(grz) = (flux(g) + 0.8×flux(r) + 0.5×flux(z)) / 2.3)—at brighter magnitudes, QSOs are
rare and stars abundant. To further reduce stellar contamination, we apply a stricter W1 − W2 cut to sources near
the stellar locus in g − r vs. r − z. We also impose a minimum signal-to-noise ratio in both WISE bands.


In [17]:
def is_QSO_target(g,r,z,maskbit):
    if r > 22.7:
        return False
    if r < 17.5:
        return False
    if not g - r < 1.3:
        return False
    if not (-0.4 < r - z < 1.1):
        return False
    return True

def is_LRG_target(g,r,z,W1):
    if not (18.01 < z < 20.41):
        return False
    if not (0.75 < (r - z) < 2.45):
        return False
    if not (-0.6 < (z - W1) - 0.8*(r - z)):
        return False
    if not ((z - 17.18)/2 < (r - z) < (z - 15.11)/2):
        return False
    if not (((r - z) > 1.15) or ((g-r) > 1.65)):
        return False
    return True
    
def is_ELG_target(g,r,z):
    if not (21.0 < g < 23.45):
        return False
    if not (0.3 < (r - z) < 1.6):
        return False
    if not ((g-r) < 1.15*(r-z) - 0.15):
        return False
    if not ((g-r) < 1.6 - 1.2*(r-z)):
        return False
    return True




## Classifying the targets: 
0 = no label,
1 = LRG,
2 = ELG,
3 = QSA 

In [18]:
#Checking for duplicate classifications --> only 200 get 1 label at all --> is this correct

target_label_array = np.zeros(len(mag_g))
for i in range(len(mag_g)):
    if is_LRG_target(mag_g[i], mag_r[i], mag_z[i], mag_w1[i]):
        target_label_array[i] = 1
        continue
    if is_ELG_target(mag_g[i], mag_r[i], mag_z[i]):
        target_label_array[i] = 2
        continue
    if is_QSO_target(mag_g[i], mag_r[i], mag_z[i], maskbits[i]):
        target_label_array[i] = 3
        continue

print("LRG: ",(target_label_array == 1).sum())
print("ELG: ",(target_label_array == 2).sum())
print("QSO: ",(target_label_array == 3).sum())





LRG:  34
ELG:  10
QSO:  177
